### <p style="color:yellow;">Descrição do problema:</p>

Dados os obstáculos enfrentados pelo sistema educacional, a evasão escolar
desempenha um papel crucial, afetando estudantes de diferentes contextos
socioeconômicos. Com base nisso, este trabalho propõe **construir um modelo
preditivo baseado em técnicas de Ciência de Dados para prever a taxa de evasão no
ensino médio e técnico brasileiro**. O estudo aborda a complexidade deste fenômeno,
destacando a sua natureza social e a necessidade de soluções proativas. A
abordagem utiliza dados do censo escolar da educação básica e inclui a análise
exploratória de dados e técnicas de aprendizado de máquina. O modelo proposto visa
prever a evasão escolar, permitindo uma intervenção personalizada e prestando
apoio específico aos alunos desfavorecidos. Além de contribuir para a compreensão
dos fatores associados à evasão escolar, os resultados pretendem oferecer uma
abordagem inovadora para a prevenção desse fenômeno, indo além do meio
acadêmico de modo a proporcionar contribuições práticas ao campo educacional.

### <p style="color:yellow;">Algoritmos a serem utilizados:</p>

- **Decision Tree**:

    - Motivo: A Decision Tree é um algoritmo intuitivo e fácil de interpretar, que funciona dividindo os dados em subgrupos baseados em perguntas de sim ou não. Cada divisão é feita de forma a maximizar a separação entre as classes, o que permite capturar padrões claros e específicos no conjunto de dados. É particularmente útil para identificar regras de decisão simples que podem explicar a evasão ou a conclusão dos estudos, tornando-o eficaz para problemas de classificação.

- **K-Nearest Neighbors (KNN)**:

    - Motivo: O KNN é um algoritmo que apesar de simples, pode ser útil para captar grupos similares entre os alunos, onde utiliza cálculos de distância para encontrar os vizinhos mais próximos a um ponto específico, atribuindo uma nova classe.

- **Logistic Regression**:

    - Motivo: É um modelo de fácil interpretação e bastante robusto e eficaz quando se trata de problemas de classificação ou quando há uma relação linear entre as carcterísticas e a probabilidade de evasão.

### Importação das bibliotecas

In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt


#from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
#from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.model_selection import  GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import OneHotEncoder,  MinMaxScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import accuracy_score,  precision_score, f1_score, recall_score,classification_report

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import inflection
import warnings
warnings.filterwarnings("ignore")

In [2]:
dados_tec = pd.read_csv('../data/processed/dadosTEC.csv')
dados_tec.head()

,Carga Horária,Categoria da Situação,Cor/Raça,Data de Início do Ciclo,Data de Fim Previsto do Ciclo,Faixa Etária,Fator Esforço Curso,Idade,Instituição,Mês de Ocorrência da Situação,...,dia_inicio_ciclo,mes_inicio_ciclo,ano_inicio_ciclo,dia_fim_ciclo,mes_fim_ciclo,ano_fim_ciclo,dia_ocorrencia,mes_ocorrencia,ano_ocorrencia,delta_days
0,3600,1,não declarada,2015-02-19,2019-02-19,15 a 19 anos,1.16,19.0,IFSUL,2019-07-01,...,19,2,2015,19,2,2019,1,7,2019,1593
1,3600,0,não declarada,2015-02-19,2019-02-19,20 a 24 anos,1.16,20.0,IFSUL,2019-08-01,...,19,2,2015,19,2,2019,1,8,2019,1624
2,3600,0,não declarada,2015-02-19,2019-02-19,15 a 19 anos,1.16,18.0,IFSUL,2019-04-01,...,19,2,2015,19,2,2019,1,4,2019,1502
3,3600,0,não declarada,2015-02-19,2019-02-19,15 a 19 anos,1.16,19.0,IFSUL,2019-04-01,...,19,2,2015,19,2,2019,1,4,2019,1502
4,3600,0,não declarada,2015-02-19,2019-02-19,20 a 24 anos,1.16,20.0,IFSUL,2019-03-01,...,19,2,2015,19,2,2019,1,3,2019,1471


In [3]:
dados_tec.shape

(166187, 26)

### Renomeando colunas

In [4]:
dados_tec.columns = dados_tec.columns.str.replace(' ','')
new_cols = []
for column in dados_tec.columns:
    new_cols.append(inflection.underscore(column))
dados_tec.columns = new_cols

In [5]:
dados_tec.rename(columns={'categoriada_situação':'categoria_da_situação',
                            'datade_iníciodo_ciclo':'data_de_inicio_do_ciclo',
                            'datade_fim_previstodo_ciclo':'data_de_fim_previsto_do_ciclo',
                            'mêsde_ocorrênciada_situação':'mês_de_ocorrência_da_situação'},inplace=True)

In [6]:
dados_tec.columns

Index(['carga_horária', 'categoria_da_situação', 'cor/raça',
       'data_de_inicio_do_ciclo', 'data_de_fim_previsto_do_ciclo',
       'faixa_etária', 'fator_esforço_curso', 'idade', 'instituição',
       'mês_de_ocorrência_da_situação', 'região', 'renda_familiar', 'sexo',
       'turno', 'uf', 'unidadede_ensino', 'dia_inicio_ciclo',
       'mes_inicio_ciclo', 'ano_inicio_ciclo', 'dia_fim_ciclo',
       'mes_fim_ciclo', 'ano_fim_ciclo', 'dia_ocorrencia', 'mes_ocorrencia',
       'ano_ocorrencia', 'delta_days'],
      dtype='object')

### Filtragem de variáveis

Existem algumas varíaveis que foram necessárias apenas para derivar outras e que não faz mais sentido mantê-las. São:
- data_de_inicio_do_ciclo
- data_de_fim_previsto_do_ciclo
- mês_de_ocorrência_da_situação

In [7]:
dados_tec.drop(columns=['data_de_inicio_do_ciclo','data_de_fim_previsto_do_ciclo',
                          'mês_de_ocorrência_da_situação'],inplace=True)

### Separação de Features e Target

In [8]:
features_tec = dados_tec.drop(columns=['categoria_da_situação'])
features_tec.head(3)

,carga_horária,cor/raça,faixa_etária,fator_esforço_curso,idade,instituição,região,renda_familiar,sexo,turno,...,dia_inicio_ciclo,mes_inicio_ciclo,ano_inicio_ciclo,dia_fim_ciclo,mes_fim_ciclo,ano_fim_ciclo,dia_ocorrencia,mes_ocorrencia,ano_ocorrencia,delta_days
0,3600,não declarada,15 a 19 anos,1.16,19.0,IFSUL,Região Sul,"0<RFP<=0,5",Feminino,Matutino,...,19,2,2015,19,2,2019,1,7,2019,1593
1,3600,não declarada,20 a 24 anos,1.16,20.0,IFSUL,Região Sul,Não declarada,Feminino,Matutino,...,19,2,2015,19,2,2019,1,8,2019,1624
2,3600,não declarada,15 a 19 anos,1.16,18.0,IFSUL,Região Sul,Não declarada,Feminino,Matutino,...,19,2,2015,19,2,2019,1,4,2019,1502


In [9]:
target_tec = dados_tec[['categoria_da_situação']]

### Separação de treino e teste

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features_tec, target_tec, test_size=0.2)

In [11]:
print(f'X_train: {X_train.shape[0]} amostras')
print(f'X_test: {X_test.shape[0]} amostras')

### Pré-processamento das variáveis

Nesta etapa iremos definir os transformadores para codificar as variáveis categóricas, bem como escalonar as variáveis numéricas.

In [12]:
categorical_features = features_tec.select_dtypes(include=['object']).columns
numerical_features = features_tec.select_dtypes(exclude=['object']).columns

In [13]:
encoder = OneHotEncoder(handle_unknown='ignore',drop='first')
minmax_scaler = MinMaxScaler()

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', minmax_scaler, numerical_features),
        ('categorical', encoder, categorical_features)
])

### Definição dos modelos

### Cross validation e Fine tuning

In [16]:
results = pd.DataFrame({})

#### KNN

In [22]:
models_results = {}
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", KNeighborsClassifier(metric="euclidean")),
    ]
)
# Inner CV
grid_search = GridSearchCV(
    pipeline,
    {
        "classifier__n_neighbors": np.arange(1, 20, 2),
        "classifier__weights": ["uniform", "distance"],
    },
    cv=5,
    scoring="accuracy",
)
# Outer CV
cross_validation_results = cross_validate(
    grid_search,
    X=features_tec,
    y=target_tec,
    cv=5,
    scoring=["accuracy", "f1", "precision", "recall"],
)

acc_scores = f"{cross_validation_results['test_accuracy'].mean():.3f} +- {cross_validation_results['test_accuracy'].std():.3f}"
f1_scores = f"{cross_validation_results['test_f1'].mean():.3f} +- {cross_validation_results['test_f1'].std():.3f}"
precision_scores = f"{cross_validation_results['test_precision'].mean():.3f} +- {cross_validation_results['test_precision'].std():.3f}"
recall_scores = f"{cross_validation_results['test_recall'].mean():.3f} +- {cross_validation_results['test_recall'].std():.3f}"

models_results["model_name"] = ["KNN"]
models_results["accuracy"] = [acc_scores]
models_results["f1"] = [f1_scores]
models_results["precision"] = [precision_scores]
models_results["recall"] = [recall_scores]

df = pd.DataFrame(models_results)
results = pd.concat([results, df])

In [23]:
results.to_csv('knn_results.csv', index=False)

#### Logistic Regression

In [25]:
models_results = {}

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(solver="liblinear", max_iter=1000)),
    ]
)
# Inner CV
grid_search = GridSearchCV(
    pipeline,
    {"classifier__penalty": ["l1", "l2"], "classifier__C": [0.1, 1.2]},
    cv=5,
    scoring="accuracy",
)
# Outer CV
cross_validation_results = cross_validate(
    grid_search,
    X=features_tec,
    y=target_tec,
    cv=5,
    scoring=["accuracy", "f1", "precision", "recall"],
)

acc_scores = f"{cross_validation_results['test_accuracy'].mean():.3f} +- {cross_validation_results['test_accuracy'].std():.3f}"
f1_scores = f"{cross_validation_results['test_f1'].mean():.3f} +- {cross_validation_results['test_f1'].std():.3f}"
precision_scores = f"{cross_validation_results['test_precision'].mean():.3f} +- {cross_validation_results['test_precision'].std():.3f}"
recall_scores = f"{cross_validation_results['test_recall'].mean():.3f} +- {cross_validation_results['test_recall'].std():.3f}"

models_results["model_name"] = ["Logistic Regression"]
models_results["accuracy"] = [acc_scores]
models_results["f1"] = [f1_scores]
models_results["precision"] = [precision_scores]
models_results["recall"] = [recall_scores]

df = pd.DataFrame(models_results)
results = pd.concat([results, df])

In [26]:
results.to_csv('lr_results.csv', index=False)

#### Arvore de decisão

In [18]:
from sklearn.tree import DecisionTreeClassifier

models_results = {}

pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier',  DecisionTreeClassifier())
    ])
# Inner CV
grid_search = GridSearchCV(pipeline, {
            "classifier__criterion": ["gini","entropy"],
            "classifier__max_depth": [10, 20, 30, 50],
            "classifier__min_samples_split": [2, 5, 10],
            "classifier__min_samples_leaf": [1, 2, 4],
        },cv=5, scoring='accuracy')
# Outer CV
cross_validation_results = cross_validate(grid_search, X=features_tec, y=target_tec,
                             cv=5, scoring=['accuracy', 'f1', 'precision', 'recall'])

acc_scores = f"{cross_validation_results['test_accuracy'].mean():.3f} +- {cross_validation_results['test_accuracy'].std():.3f}"
f1_scores = f"{cross_validation_results['test_f1'].mean():.3f} +- {cross_validation_results['test_f1'].std():.3f}"
precision_scores = f"{cross_validation_results['test_precision'].mean():.3f} +- {cross_validation_results['test_precision'].std():.3f}"
recall_scores = f"{cross_validation_results['test_recall'].mean():.3f} +- {cross_validation_results['test_recall'].std():.3f}"

models_results['model_name'] = ['Arvore de decisão']
models_results['accuracy'] = [acc_scores]
models_results['f1'] = [f1_scores]
models_results['precision'] = [precision_scores]
models_results['recall'] = [recall_scores]

df = pd.DataFrame(models_results)
results = pd.concat([results, df])

In [19]:
results.to_csv('dt_results.csv', index=False)

In [ ]:
results

,model_name,accuracy,f1,precision,recall
0,Arvore de decisão,0.866 +- 0.037,0.760 +- 0.076,0.886 +- 0.122,0.699 +- 0.156
0,KNN,0.716 +- 0.067,0.485 +- 0.078,0.598 +- 0.157,0.422 +- 0.069
0,Logistic Regression,0.846 +- 0.029,0.702 +- 0.094,0.872 +- 0.084,0.621 +- 0.177


In [21]:
# salvando o modelo arvore de decisão
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

### Conclusões


O melhor modelo foi a Arvore de decisão. Explicação:

> **Recall:**
-  No problema da evasão estudantil, o foco principal é identificar todos os alunos em risco, que significa maximizar o recall, uma vez que essa métrica nos dá a probabilidade de uma classe positiva ser corretamente classificada. O recall do modelo no conjunto de teste foi de 90%, garantindo que poucos alunos evadidos fossem deixados de fora, isto é, 90% dos evadidos foram corretamente classificados pelo modelo.

> **Precisão:**
- Essa também é uma métrica importante se quisermos priorizar alarmes falsos, que seriam os casos de falsos positivos - quando o aluno não é evadido, mas o modelo classifica como tal. O modelo apresentou uma precisão de 94%, ou seja, de todos os alunos que ele classificou como evadido, ele esteve correto em 94% das vezes. Isso garante que esforços de intervenção não sejam desperdiçados em alunos que não estejam em risco.